In [1]:
import pandas as pd
import numpy as np
import requests
import json


In [ ]:
url = 'http://api.open-notify.org/iss-now.json'
iss_response = requests.get(url)


reviews = pd.read_csv('rt.reviews.tsv.gz')
info = pd.read_csv('Rt.movie_info.tsv.gz') 